In [17]:
import pandas as pd

df = pd.read_csv("24hu_articles.csv", sep="|", dtype='str')

In [22]:
df

,url,page,title,content,date,full_text
0,https://24.hu/fn/gazdasag/2025/01/07/epitoipar...,1,Sürgősen biztosítást kell kötniük az építőipar...,"Aki az építőiparban dolgozik, annak kötelező k...",2025/01/07,Sürgősen biztosítást kell kötniük az építőipar...
1,https://24.hu/fn/gazdasag/2025/01/07/bekescsab...,1,Milliókat tehetett zsebre a tanárok pénzéből a...,15–20 embert is megkárosíthatott éveken át a B...,2025/01/07,Milliókat tehetett zsebre a tanárok pénzéből a...
2,https://24.hu/fn/gazdasag/2025/01/07/mbh-otp-f...,1,Több mint 43 milliós felügyeleti bírságot kapo...,Több mint 43 millió forint bírságot szabott ki...,2025/01/07,Több mint 43 milliós felügyeleti bírságot kapo...
3,https://24.hu/fn/gazdasag/2025/01/07/nagy-mart...,1,Nagy Márton 15 millió forintból repült tovább ...,Nagy Márton hivatalos külföldi utazásainak ára...,2025/01/07,Nagy Márton 15 millió forintból repült tovább ...
4,https://24.hu/fn/gazdasag/2025/01/07/gazszamla...,1,"Ha ilyen gázszámlát kapott, ne hagyja szó nélk...",Furcsa éves földgázelszámoló számlát kapott né...,2025/01/07,"Ha ilyen gázszámlát kapott, ne hagyja szó nélk..."
...,...,...,...,...,...,...
15785,https://24.hu/fn/gazdasag/2017/12/08/vosz-vita...,659,"VOSZ: vita nincs, muszáj a létszámleépítés a k...",Kapcsolódó cikkek Ki a kolonc? Berágott az ÉSZ...,2017/12/08,"VOSZ: vita nincs, muszáj a létszámleépítés a k..."
15786,https://24.hu/fn/gazdasag/2017/12/08/gondok-le...,659,Gondok lehetnek az építőiparban,Kapcsolódó cikkek Varga Mihály: nem lesz hossz...,2017/12/08,Gondok lehetnek az építőiparban. Kapcsolódó ci...
15787,https://24.hu/fn/gazdasag/2017/12/08/kongatjak...,659,Kongatják a vészharangot: túlhevülhetett a mag...,Kapcsolódó cikkek A kormány csalódhat: nem val...,2017/12/08,Kongatják a vészharangot: túlhevülhetett a mag...
15788,https://24.hu/fn/gazdasag/2017/12/08/magannyug...,659,Magánnyugdíjvagyon: a 2826 milliárd forinttal ...,Kapcsolódó cikkek Nyugdíjvagyon: 2826 milliárd...,2017/12/08,Magánnyugdíjvagyon: a 2826 milliárd forinttal ...


In [21]:
df = df[(df["title"] != "Title not found") & ~df["content"].str.contains("Content not found") & ~df["content"].str.strip().isna() & ~df["content"].str.contains("Error fetching content")]

In [24]:
df = df.reset_index(drop=True)

In [25]:
df.loc[24,"content"]

'Az orosz állami energiavállalat, a Gazprom leállította az Ukrajnán keresztül érkező földgáz szállítását – írja a New York Times a Gazprom közlése alapján. Az ok, hogy a tranzitszolgáltatásra vonatkozó 2019-es egyezményt nem újították meg, így az szerdán megszűnt. Volodimir Zelenszkij ukrán elnök már többször fenyegetőzött azzal, hogy lekapcsolja a Magyarországot, Szlovákiát és néhány balkáni országot kiszolgáló Barátság vezetéket, azonban ez a téma nem került szóba az újévi beszédében. A döntést azonban a jelek szerint mégsem ő hozta meg, hanem a Gazprom volt az, aki lépett, moszkvai idő szerint reggel 8 órakor. Az érintett országok közül Moldova a leginkább kiszolgáltatott, decemberben rendkívüli állapotot is hirdettek attól tartva, hogy az orosz gáz Ukrajnán keresztül történő szállításának megszüntetése veszélyeztetné fő áramforrását, egy gázüzemű erőművet. A témában kérdéseket küldtünk a külügyminisztériumnak, amint válaszolnak, frissítjük cikkünket.'

In [21]:
texts = [df.loc[2,"content"],df.loc[3,"content"]]

In [3]:
df['full_text'] = df['title'] + ". " + df['content']

In [5]:
df.to_csv("24hu_articles.csv", sep="|", encoding='utf-8', index=False)

In [9]:
import pandas as pd
from google import genai
from google.genai import types
import time

client = genai.Client(api_key="")

# Function to translate each text
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Load existing results if available (to resume progress)
try:
    df_translated = pd.read_csv("translated_articles_24hu.csv")
    processed_ids = set(df_translated.index)
except FileNotFoundError:
    df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
    processed_ids = set()

# Batch processing with periodic saving
batch_size = 50  # Number of rows to process per batch
start_index = max(processed_ids) + 1 if processed_ids else 0

for i in range(start_index, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    
    # Translate each article in the batch
    batch["translated_content"] = batch["full_text"].apply(translate_text)
    
    # Append to the result DataFrame
    df_translated = pd.concat([df_translated, batch], ignore_index=True)
    
    # Save to CSV after each batch
    df_translated.to_csv("translated_articles_24hu.csv", index=False, sep="|", encoding='utf-8')
    
    print(f"Batch {i // batch_size + 1} processed and saved. Total articles translated: {len(df_translated)}")
    time.sleep(2)  # Add a small delay to avoid hitting API rate limits

C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\632321810.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


Batch 1 processed and saved. Total articles translated: 50


C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\632321810.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


Batch 2 processed and saved. Total articles translated: 100


C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\632321810.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


...

C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\632321810.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


Batch 168 processed and saved. Total articles translated: 8400
Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}
Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\632321810.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = batch["full_text"].apply(translate_text)


Batch 169 processed and saved. Total articles translated: 8450
Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


KeyboardInterrupt: 

In [23]:
import pandas as pd
import asyncio
from concurrent.futures import ThreadPoolExecutor
from google import genai
from google.genai import types

client = genai.Client(api_key="")

# Function to translate each text
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main processing loop
async def main(df):
    try:
        df_translated = pd.read_csv("translated_articles_24hu.csv", sep="|", encoding='utf-8')
        processed_ids = set(df_translated.index)
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
        processed_ids = set()

    batch_size = 75  # Number of rows to process per batch
    start_index = max(processed_ids) + 1 if processed_ids else 0

    for i in range(start_index, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        batch = await translate_batch_async(batch)
        
        # Append to the result DataFrame
        df_translated = pd.concat([df_translated, batch], ignore_index=True)
        
        # Save to CSV after each batch
        df_translated.to_csv("translated_articles_24hu.csv", index=False, sep="|", encoding='utf-8')
        
        print(f"Batch {i // batch_size + 1} processed and saved. Total articles translated: {len(df_translated)}")


# Run the asynchronous main function
await main(df)


C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\2665177933.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 113 processed and saved. Total articles translated: 8525


C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\2665177933.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 114 processed and saved. Total articles translated: 8600


C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\2665177933.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


...

Batch 208 processed and saved. Total articles translated: 15650


C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\2665177933.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 209 processed and saved. Total articles translated: 15725
Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}


C:\Users\tothk\AppData\Local\Temp\ipykernel_6364\2665177933.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Batch 210 processed and saved. Total articles translated: 15790


In [13]:
!pip install nest_asyncio

In [8]:
import pandas as pd

df_proba = pd.read_csv("translated_articles_24hu.csv", sep="|", dtype='str')

In [9]:
df_proba[df_proba["translated_content"].isnull()].count()

full_text             0
translated_content    0
url                   0
page                  0
title                 0
content               0
date                  0
dtype: int64

In [6]:
import pandas as pd
import asyncio
from concurrent.futures import ThreadPoolExecutor
from google import genai
from google.genai import types

client = genai.Client(api_key="")


# Function to translate each text 
def translate_text(text):
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
                system_instruction="You are a translator that translates Hungarian text to English."
            ),
            contents=f"Please translate the following article to English in simple text format: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Asynchronous wrapper for the translate function
async def translate_batch_async(batch):
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        tasks = [loop.run_in_executor(executor, translate_text, text) for text in batch["full_text"]]
        results = await asyncio.gather(*tasks)
    batch["translated_content"] = results
    return batch

# Main function to process and retry NaN translations
async def main():
    try:
        df_translated = pd.read_csv("translated_articles_24hu.csv", sep="|", encoding='utf-8')
    except FileNotFoundError:
        df_translated = pd.DataFrame(columns=["full_text", "translated_content"])
    
    # First pass: Process untranslated rows
    untranslated_rows = df_translated["translated_content"].isna().sum()
    print(f"Untranslated rows found: {untranslated_rows}")

    if untranslated_rows > 0:
        to_retry = df_translated[df_translated["translated_content"].isna()]
        df_translated.loc[to_retry.index, "translated_content"] = (await translate_batch_async(to_retry))["translated_content"].values

        # Save the corrected translations
        df_translated.to_csv("translated_articles_24hu.csv", index=False, sep="|", encoding='utf-8')
        print(f"Retry complete! Remaining untranslated rows: {df_translated['translated_content'].isna().sum()}")
    
    print("Translation process complete!")

# Run the asynchronous main function
await main()


Untranslated rows found: 89


C:\Users\tothk\AppData\Local\Temp\ipykernel_10924\3280305730.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch["translated_content"] = results


Retry complete! Remaining untranslated rows: 78
Translation process complete!


In [7]:
df_proba = pd.read_csv("translated_articles_24hu.csv", sep="|", dtype='str')

df_proba.dropna(subset=['translated_content'], inplace=True)
df_proba.reset_index(drop=True, inplace=True)

df_proba.to_csv("translated_articles_24hu.csv", index=False, sep="|", encoding='utf-8')